# **Imports and defining variables**

In [1]:
import pandas as pd
import re
import requests
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from random import randint
from time import sleep
from io import StringIO

ModuleNotFoundError: No module named 'bs4'

In [ ]:
navigator = 'Microsoft Edge (Scrapping; This is my system)'
url_allbrands_homepage = 'https://www.auto-data.net/en/allbrands'
url = 'https://www.auto-data.net'

In [ ]:
html = requests.get(url_allbrands_homepage, headers={'User-Agent': navigator})
html

# **Scraping**

## Creating the soup and cleaning the information to get only text

In [18]:
soup = BeautifulSoup(html.text, 'html.parser')

In [19]:
brands = soup.find_all('a', {'class' : 'marki_blok'})
brands[0:5]

[<a class="marki_blok" href="/en/abarth-brand-200" title="Abarth - Technical Specs, Fuel consumption, Dimensions"><img alt="Abarth - Technical Specs, Fuel consumption, Dimensions" src="/img/logos/Abarth.png"/><strong>Abarth</strong></a>,
 <a class="marki_blok" href="/en/ac-brand-1" title="AC - Technical Specs, Fuel consumption, Dimensions"><img alt="AC - Technical Specs, Fuel consumption, Dimensions" src="/img/logos/AC.png"/><strong>AC</strong></a>,
 <a class="marki_blok" href="/en/acura-brand-6" title="Acura - Technical Specs, Fuel consumption, Dimensions"><img alt="Acura - Technical Specs, Fuel consumption, Dimensions" src="/img/logos/Acura.png"/><strong>Acura</strong></a>,
 <a class="marki_blok" href="/en/aiways-brand-301" title="Aiways - Technical Specs, Fuel consumption, Dimensions"><img alt="Aiways - Technical Specs, Fuel consumption, Dimensions" src="/img/logos/Aiways.png"/><strong>Aiways</strong></a>,
 <a class="marki_blok" href="/en/aixam-brand-255" title="Aixam - Technical Sp

In [20]:
#This loop is to get only the part to use in link ie whatever is after 'href='
brand_list = re.findall('href="(\S+)"',str(brands))
brand_list[0:5]

['/en/abarth-brand-200',
 '/en/ac-brand-1',
 '/en/acura-brand-6',
 '/en/aiways-brand-301',
 '/en/aixam-brand-255']

In [21]:
#Looping to remove the /en since the url already has it and removing it makes it so it will default to another language
brand_list_final = []
for i in range(len(brand_list)):
    brand_list_final += re.findall('en/(\S+)', brand_list[i])
brand_list_final[0:5]
#Starting to create dataframe for the links
df = pd.DataFrame(brand_list_final)
df['letter'] = df[0].apply(lambda x: x[0])
df['letter'].unique()

array(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'z'],
      dtype=object)

In [22]:
#Creating CSV files for each starting letter of the brand names so we can save it more often
dicto = {}
for letter in df['letter'].unique():
    a = df[df['letter'] == letter].drop(columns = 'letter')
    a.to_csv('../BEEV/letter_csvs/' + letter + '.csv')
    dicto[letter] = a

## Creating the loops for the actual scraping

In [23]:
#Looping to create a column with url brand names
url = 'https://www.auto-data.net/en/'
for csvs in dicto.values():
    csvs[0] = csvs[0].apply(lambda x: url + x)

In [24]:
#Renaming the column to make it easier to understand
for csvs in dicto.values():
    csvs.rename(columns={0 : 'brands_url'}, inplace = True)

In [25]:
#Dataframe with url for each brand
for letter, letter_csvs in dicto.items():
    letter_csvs.to_csv('../BEEV/letter_csvs/' + letter + '.csv')

In [26]:
#Function to get the models inside each brand
def model_of_each_brand(brand_url):
    models_list = []
    html2 = requests.get(brand_url, headers={'User-Agent': navigator})
    soup2 = BeautifulSoup(html2.text, 'html.parser')
    models = soup2.find_all('a', {'class' : 'modeli'}) 
    models_list_final = re.findall('en/(\S+)', str(models))
    return [x[:-1] for x in models_list_final]
brand_url = 'https://www.auto-data.net/en/abarth-brand-200'
model_of_each_brand(brand_url)

['abarth-124-spider-model-2152',
 'abarth-500-model-2872',
 'abarth-595-model-2149',
 'abarth-695-model-2150']

In [27]:
#Applying the function to get the models for each brand(displayed in lists inside the model_url column)
for csvs in dicto.values():
    csvs['models'] = csvs['brands_url'].apply(lambda x: model_of_each_brand(x))


In [28]:
#Saving to keep a column with models names
for letter, letter_csvs in dicto.items():
    letter_csvs.to_csv('../BEEV/letter_csvs/' + letter + '.csv')

In [29]:
#Loop to explode the models name and get 1 name per each line
for letter, letter_csvs in dicto.items():
    dicto[letter] = dicto[letter].explode('models')

In [30]:
dicto['a'].head()         #Just checking

,brands_url,models
0,https://www.auto-data.net/en/abarth-brand-200,abarth-124-spider-model-2152
0,https://www.auto-data.net/en/abarth-brand-200,abarth-500-model-2872
0,https://www.auto-data.net/en/abarth-brand-200,abarth-595-model-2149
0,https://www.auto-data.net/en/abarth-brand-200,abarth-695-model-2150
1,https://www.auto-data.net/en/ac-brand-1,ac-ace-model-1


In [31]:
#Loop to create a column with models_url
for csvs in dicto.values():
    csvs['models_url'] = csvs['models'].apply(lambda x: url + x)

In [32]:
#Saving the models_url column
for letter, letter_csvs in dicto.items():
    letter_csvs.to_csv('../BEEV/letter_csvs/' + letter + '.csv')

In [33]:
#Creating function to scrap the generation names for each models
def generation_of_each_model(model_url):
    gens_list = []
    html3 = requests.get(model_url, headers={'User-Agent': navigator})
    soup3 = BeautifulSoup(html3.text, 'html.parser')
    gens = soup3.find_all('a', {'class' : 'position'}) 
    gens_list_final = re.findall('en/(\S+)', str(gens))
    return [x[:-1] for x in gens_list_final]
model_url = 'https://www.auto-data.net/en/abarth-124-spider-model-2152'
generation_of_each_model(model_url)

['abarth-124-gt-generation-6774',
 'abarth-124-gt-generation-6774',
 'abarth-124-spider-generation-5004',
 'abarth-124-spider-generation-5004']

In [34]:
#Applying function to get the generation names for each models
for csvs in dicto.values():
    csvs['generations'] = csvs['models_url'].apply(lambda x: generation_of_each_model(x))

In [35]:
#Saving the generations columns
for letter, letter_csvs in dicto.items():
    letter_csvs.to_csv('../BEEV/letter_csvs/' + letter + '.csv')

In [36]:
#Loop to explode the generations names
for letter, letter_csvs in dicto.items():
    dicto[letter] = dicto[letter].explode('generations')

In [37]:
#Saving the exploded generations names
for letter, letter_csvs in dicto.items():
    letter_csvs.to_csv('../BEEV/letter_csvs/' + letter + '.csv')

In [38]:
#Dropping duplicates that were created
for letter, letter_csvs in dicto.items():
    dicto[letter].drop_duplicates(inplace = True)
    letter_csvs.to_csv('../BEEV/letter_csvs/' + letter + '.csv')

In [39]:
#Looping to create a column for the generations url
for csvs in dicto.values():
    csvs['generations_url'] = csvs['generations'].apply(lambda x: url + x)

In [40]:
#Saving the column with the generations urls
for letter, letter_csvs in dicto.items():
    letter_csvs.to_csv('../BEEV/letter_csvs/' + letter + '.csv')

In [41]:
#Creating a function to scrap for the modifications of each generation
def modifications_of_each_generation(gen_url):
    mods_list = []
    html4 = requests.get(gen_url, headers={'User-Agent': navigator})
    soup4 = BeautifulSoup(html4.text, 'html.parser')
    mods = soup4.find_all('table', {'class' : 'carlist'}) 
    gens_list_final = re.findall('en/(\S+)', str(mods))
    return [x[:-1] for x in gens_list_final]
gen_url = 'https://www.auto-data.net/en/alfa-romeo-mito-generation-363'
modifications_of_each_generation(gen_url)          

['alfa-romeo-mito-1.6-jtdm-120hp-16680',
 'alfa-romeo-mito-1.6-jtdm-120hp-16680',
 'alfa-romeo-mito-1.4-tb-multiair-170hp-44386',
 'alfa-romeo-mito-1.4-tb-multiair-170hp-44386',
 'alfa-romeo-mito-1.4-tb-155hp-16679',
 'alfa-romeo-mito-1.4-tb-155hp-16679',
 'alfa-romeo-mito-1.4-tb-multiair-135hp-tct-44377',
 'alfa-romeo-mito-1.4-tb-multiair-135hp-tct-44377',
 'alfa-romeo-mito-1.4-t-jet-120hp-16683',
 'alfa-romeo-mito-1.4-t-jet-120hp-16683',
 'alfa-romeo-mito-1.4-multiair-135hp-start-stop-16681',
 'alfa-romeo-mito-1.4-multiair-135hp-start-stop-16681',
 'alfa-romeo-mito-1.4-multiair-105hp-start-stop-16684',
 'alfa-romeo-mito-1.4-multiair-105hp-start-stop-16684',
 'alfa-romeo-mito-1.4-mpi-78hp-16682',
 'alfa-romeo-mito-1.4-mpi-78hp-16682',
 'alfa-romeo-mito-1.4-70hp-44378',
 'alfa-romeo-mito-1.4-70hp-44378',
 'alfa-romeo-mito-1.3-jtdm-90hp-40499',
 'alfa-romeo-mito-1.3-jtdm-90hp-40499',
 'alfa-romeo-mito-1.3-jtdm-eco-85hp-44387',
 'alfa-romeo-mito-1.3-jtdm-eco-85hp-44387',
 'alfa-romeo-mit

In [42]:
#Applying function to get the modifications names for each generation
for csvs in dicto.values():
    csvs['modifications'] = csvs['generations_url'].apply(lambda x: modifications_of_each_generation(x))

In [43]:
#Saving the mods column
for letter, letter_csvs in dicto.items():
    letter_csvs.to_csv('../BEEV/letter_csvs/' + letter + '.csv')

In [44]:
#Exploding the mods column to get one per row
for letter, letter_csvs in dicto.items():
    dicto[letter] = dicto[letter].explode('modifications')

In [45]:
#Saving the exploded generations names
for letter, letter_csvs in dicto.items():
    letter_csvs.to_csv('../BEEV/letter_csvs/' + letter + '.csv')

In [46]:
#Dropping duplicates that were created
for letter, letter_csvs in dicto.items():
    dicto[letter].drop_duplicates(inplace = True)
    letter_csvs.to_csv('../BEEV/letter_csvs/' + letter + '.csv')

In [47]:
#Loop to create a column for the mods url
for csvs in dicto.values():
    csvs['modifications_url'] = csvs['modifications'].apply(lambda x: url + str(x))

In [177]:
#Saving the column for the mods urls
for letter, letter_csvs in dicto.items():
    letter_csvs.to_csv('../BEEV/letter_csvs/' + letter + '.csv')

In [49]:
#This is the final loop to get the details for the cars
#specs = soup4.find_all('table', {'class' : 'cardetailsout car2'})

In [50]:
#Need to install this library to work
!pip install html5lib

## Final dataframe

In [36]:
#Creating the final function 

def specifications_of_each_car(mods_url):
    print(mods_url)
    if mods_url[-3:] == 'nan':
        return pd.DataFrame()
    else:
        specs = pd.read_html(mods_url)
        if len(specs)>=2:
            i=1
        elif len(specs)==1:
            return specs[0].T
        else:
            return pd.DataFrame()
        specs = specs[i]
        #print(specs.T)
        #specs2 = specs[(specs['General information'] == 'Brand')
        #|(specs['General information'] == 'Model')
        #|(specs['General information'] == 'Generation')
        #|(specs['General information'] == 'Modification (Engine)')
        #|(specs['General information'] == 'CO2 emissions') 
        #|(specs['General information'] == 'Body type')
        #|(specs['General information'] == 'Fuel tank capacity')
        #|((specs['General information'] == 'Fuel consumption (economy) - combined') | (specs['General information'] == 'Fuel consumption (economy) - combined (NEDC)'))
        #|(specs['General information'] == 'Fuel Type')
        #|(specs['General information'] == 'Power')]
        specs.rename(columns = {'General information' : ''}, inplace = True)
        specs.set_index([''], inplace = True)
        #specs3 = specs2.T
        #specs3.reset_index(inplace = True)
        #specs3.drop(columns = ['index'], inplace = True)
        #specs3['Fuel tank(l)'] = specs3['Fuel tank capacity'].apply(lambda x : x.split(' ')[0])
        #specs3['Fuel consumption(l/100km)'] = specs3['Fuel consumption (economy) - combined'].apply(lambda x : x.split(' ')[0])
        #specs3['Power(Hp)'] = specs3['Power'].apply(lambda x: x.split(' ')[0])
        #specs3['Range (Km)'] = specs3[['Fuel tank(l)','Fuel consumption(l/100km)']].apply(lambda item: int(item[0])/float(item[1])*100,axis=1)
        #specs3.drop(columns = ['Fuel consumption (economy) - combined', 'Fuel tank capacity', 'Power'], inplace = True)
        return specs.T
    

specifications_of_each_car('https://www.auto-data.net/en/abarth-124-gt-1.4-multiair-170hp-automatic-35172')

https://www.auto-data.net/en/abarth-124-gt-1.4-multiair-170hp-automatic-35172


,Brand,Model,Generation,Modification (Engine),Start of production,Powertrain Architecture,Body type,Seats,Doors,Performance specs,...,Front suspension,Rear suspension,Front brakes,Rear brakes,Assisting systems,Steering type,Power steering,Tires size,Wheel rims size,(adsbygoogle = window.adsbygoogle || []).push({});
General information.1,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp) Automatic,2018 year,Internal Combustion engine,Roadster,2,2,Performance specs,...,Double wishbone,Multi-link independent,Ventilated discs,Disc,ABS (Anti-lock braking system),Steering rack and pinion,Electric Steering,205/45 R17 W84,17,(adsbygoogle = window.adsbygoogle || []).push(...


In [97]:
#Creating an empty dataframe
df = pd.DataFrame(columns = ['Brand', 'Model', 'Generation', 'Modification(Engine)','Body type', 'Fuel consumption (economy) - combined', 'CO2 emissions', 'Fuel type','Power','Fuel tank capacity', 'Fuel tank(l)', 'Fuel consumption(l/100km)', 'Power(Hp)','Range (km)'])
df.reset_index()


,index,Brand,Model,Generation,Modification(Engine),Body type,Fuel consumption (economy) - combined,CO2 emissions,Fuel type,Power,Fuel tank capacity,Fuel tank(l),Fuel consumption(l/100km),Power(Hp),Range (km)


In [182]:
#Testing for 'a' dataframe:
#for csvs in dicto['a']:
#    specifications_of_each_car(csvs['modifications_url'])
 #   df = pd.concat([df,specs3], ignore_index=True)

#Taking the shape to loop
#dicto['a']['modifications_url'].shape[0]
#dicto['b'].reset_index(inplace = True)
#global specs3   
#for i in range(0,dicto['a']['modifications_url'].shape[0]+1):
#    result = specifications_of_each_car(dicto['a']['modifications_url'].iloc[i])
#    print(i)
#    result = pd.concat([df,result], ignore_index=True)
#df.head()

In [55]:
for letter in dicto.keys():
    dicto[letter]['dataframe'] = dicto[letter]['modifications_url'].apply(specifications_of_each_car)
    #df_car = pd.concat(list(dicto[letter]['dataframe']),ignore_index=True)
    pd.concat(list(dicto[letter]['dataframe']),ignore_index=True,join = 'inner').to_csv(letter + '_scraped.csv',index=False)
    #df_car.to_csv(letter + '_scraped.csv',index=False)

https://www.auto-data.net/en/abarth-124-gt-1.4-multiair-170hp-automatic-35172
https://www.auto-data.net/en/abarth-124-gt-1.4-multiair-170hp-35171
https://www.auto-data.net/en/abarth-124-spider-1.4-multiair-170hp-automatic-24535
https://www.auto-data.net/en/abarth-124-spider-1.4-multiair-170hp-25192
https://www.auto-data.net/en/abarth-500c-1.4-t-jet-140hp-mta-42379
https://www.auto-data.net/en/abarth-500c-1.4-t-jet-135hp-42380
https://www.auto-data.net/en/abarth-500-1.4-t-jet-140hp-mta-42424
https://www.auto-data.net/en/abarth-500-1.4-t-jet-140hp-mta-42426
https://www.auto-data.net/en/abarth-500-1.4-t-jet-135hp-42425
https://www.auto-data.net/en/abarth-500-1.4-t-jet-135hp-42423
https://www.auto-data.net/en/abarth-595c-facelift-2016-turismo-1.4-t-jet-165hp-automatic-24780
https://www.auto-data.net/en/abarth-595c-facelift-2016-turismo-1.4-t-jet-165hp-25191
https://www.auto-data.net/en/abarth-595c-facelift-2016-pista-1.4-t-jet-160hp-automatic-34144
https://www.auto-data.net/en/abarth-595c-

https://www.auto-data.net/en/acura-mdx-iii-facelift-2017-3.5-v6-290hp-awd-automatic-32184
https://www.auto-data.net/en/acura-mdx-iii-facelift-2017-3.5-v6-290hp-automatic-32183
https://www.auto-data.net/en/acura-mdx-iii-facelift-2017-3.0-v6-321hp-hybrid-sh-awd-automatic-32166
https://www.auto-data.net/en/acura-mdx-iii-3.5-v6-290hp-awd-automatic-32191
https://www.auto-data.net/en/acura-mdx-iii-3.5-v6-290hp-automatic-32190
https://www.auto-data.net/en/acura-mdx-ii-3.7-v6-304hp-1080
https://www.auto-data.net/en/acura-mdx-3.5-i-v6-24v-263hp-1084
https://www.auto-data.net/en/acura-mdx-3.5-i-v6-24v-256hp-1083
https://www.auto-data.net/en/acura-mdx-3.5-i-v6-24v-243hp-1082
https://www.auto-data.net/en/acura-mdx-3.5-i-v6-24v-228hp-1081
https://www.auto-data.net/en/acura-nsx-ii-type-s-3.5-v6-600hp-hybrid-sh-awd-dct-44286
https://www.auto-data.net/en/acura-nsx-ii-3.5-v6-573hp-hybrid-sh-awd-30474
https://www.auto-data.net/en/acura-nsx-i-facelift-2002-3.2-v6-290hp-42139
https://www.auto-data.net/en/

https://www.auto-data.net/en/alfa-romeo-145-930-facelift-1997-1.6-t.-spark-120hp-1235
https://www.auto-data.net/en/alfa-romeo-145-930-facelift-1997-1.4-t.-spark-103hp-1233
https://www.auto-data.net/en/alfa-romeo-145-930-2.0-t.-spark-quadrifoglio-150hp-1242
https://www.auto-data.net/en/alfa-romeo-145-930-1.9-td-90hp-1240
https://www.auto-data.net/en/alfa-romeo-145-930-1.7-129hp-1236
https://www.auto-data.net/en/alfa-romeo-145-930-1.6-103hp-1234
https://www.auto-data.net/en/alfa-romeo-145-930-1.4-90hp-1232
https://www.auto-data.net/en/alfa-romeo-146-930-facelift-1999-2.0-ti-155hp-42162
https://www.auto-data.net/en/alfa-romeo-146-930-facelift-1999-1.9-jtd-105hp-16686
https://www.auto-data.net/en/alfa-romeo-146-930-facelift-1999-1.8-t.-spark-16v-144hp-42161
https://www.auto-data.net/en/alfa-romeo-146-930-facelift-1999-1.6-t.-spark-16v-120hp-42160
https://www.auto-data.net/en/alfa-romeo-146-930-facelift-1999-1.4-t.-spark-16v-103hp-42159
https://www.auto-data.net/en/alfa-romeo-146-930-faceli

https://www.auto-data.net/en/alfa-romeo-156-932-2.5-i-v6-24v-190hp-1491
https://www.auto-data.net/en/alfa-romeo-156-932-2.5-i-v6-24v-190hp-1490
https://www.auto-data.net/en/alfa-romeo-156-932-2.4-jtd-150hp-1489
https://www.auto-data.net/en/alfa-romeo-156-932-2.4-jtd-136hp-1488
https://www.auto-data.net/en/alfa-romeo-156-932-2.0-jts-165hp-1487
https://www.auto-data.net/en/alfa-romeo-156-932-2.0-i-16v-t.spark-155hp-1486
https://www.auto-data.net/en/alfa-romeo-156-932-1.9-jtd-115hp-1485
https://www.auto-data.net/en/alfa-romeo-156-932-1.9-jtd-105hp-1484
https://www.auto-data.net/en/alfa-romeo-156-932-1.8-16v-t.s.-144hp-1483
https://www.auto-data.net/en/alfa-romeo-156-932-1.6-16v-t.s.-120hp-1482
https://www.auto-data.net/en/alfa-romeo-159-sportwagon-3.2-jts-v6-260hp-q4-q-tronic-1536
https://www.auto-data.net/en/alfa-romeo-159-sportwagon-3.2-jts-v6-260hp-q4-1535
https://www.auto-data.net/en/alfa-romeo-159-sportwagon-3.2-jts-v6-260hp-41197
https://www.auto-data.net/en/alfa-romeo-159-sportwago

https://www.auto-data.net/en/alfa-romeo-33-907a-1.5-105hp-1377
https://www.auto-data.net/en/alfa-romeo-33-907a-1.4-i.e.-90hp-1376
https://www.auto-data.net/en/alfa-romeo-33-907a-1.4-i.e.-88hp-1375
https://www.auto-data.net/en/alfa-romeo-33-sport-wagon-905a-1.7-i.e.-105hp-4x4-1421
https://www.auto-data.net/en/alfa-romeo-33-sport-wagon-905a-1.7-i.e.-105hp-1420
https://www.auto-data.net/en/alfa-romeo-33-sport-wagon-905a-1.7-118hp-1419
https://www.auto-data.net/en/alfa-romeo-33-sport-wagon-905a-1.5-90hp-4x4-1417
https://www.auto-data.net/en/alfa-romeo-33-sport-wagon-905a-1.5-90hp-1415
https://www.auto-data.net/en/alfa-romeo-33-sport-wagon-905a-1.5-105hp-4x4-1418
https://www.auto-data.net/en/alfa-romeo-33-sport-wagon-905a-1.5-105hp-1414
https://www.auto-data.net/en/alfa-romeo-33-sport-wagon-905a-1.5-102hp-4x4-1416
https://www.auto-data.net/en/alfa-romeo-33-sport-wagon-905a-1.5-102hp-1413
https://www.auto-data.net/en/alfa-romeo-33-905-1.8-td-73hp-1412
https://www.auto-data.net/en/alfa-romeo-

...

https://www.auto-data.net/en/audi-r8-ii-spyder-4s-facelift-2019-performance-5.2-fsi-v10-570hp-rwd-s-tronic-44880
https://www.auto-data.net/en/audi-r8-ii-spyder-4s-facelift-2019-5.2-fsi-v10-570hp-quattro-s-tronic-36568
https://www.auto-data.net/en/audi-r8-ii-spyder-4s-facelift-2019-5.2-fsi-v10-540hp-rwd-s-tronic-37994
https://www.auto-data.net/en/audi-r8-ii-coupe-4s-facelift-2019-performance-5.2-fsi-v10-620hp-quattro-s-tronic-36967
https://www.auto-data.net/en/audi-r8-ii-coupe-4s-facelift-2019-performance-5.2-fsi-v10-570hp-rwd-s-tronic-44879
https://www.auto-data.net/en/audi-r8-ii-coupe-4s-facelift-2019-5.2-fsi-v10-570hp-quattro-s-tronic-36569
https://www.auto-data.net/en/audi-r8-ii-coupe-4s-facelift-2019-5.2-fsi-v10-540hp-rwd-s-tronic-37993
https://www.auto-data.net/en/audi-r8-ii-lms-facelift-2019-gt3-5.2-v10-585hp-34624
https://www.auto-data.net/en/audi-r8-ii-lms-2016-gt4-v10-495hp-s-tronic-28819
https://www.auto-data.net/en/audi-r8-ii-lms-2016-5.2-fsi-v10-585hp-27273
https://www.auto

https://www.auto-data.net/en/audi-s3-cabriolet-8v-facelift-2016-2.0-tfsi-310hp-quattro-s-tronic-23713
https://www.auto-data.net/en/audi-s3-cabriolet-8v-facelift-2016-2.0-tfsi-300hp-quattro-s-tronic-35170
https://www.auto-data.net/en/audi-s3-sportback-8v-2.0-tfsi-300hp-quattro-s-tronic-19023
https://www.auto-data.net/en/audi-s3-sportback-8v-2.0-tfsi-300hp-quattro-19022
https://www.auto-data.net/en/audi-s3-8v-2.0-tfsi-300hp-quattro-s-tronic-19018
https://www.auto-data.net/en/audi-s3-8v-2.0-tfsi-300hp-quattro-19016
https://www.auto-data.net/en/audi-s3-sedan-8v-2.0-tfsi-300hp-quattro-s-tronic-19030
https://www.auto-data.net/en/audi-s3-sedan-8v-2.0-tfsi-300hp-quattro-19029
https://www.auto-data.net/en/audi-s3-cabriolet-8v-2.0-tfsi-300hp-quattro-s-tronic-22386
https://www.auto-data.net/en/audi-s3-sportback-8pa-2.0-tfsi-265hp-quattro-s-tronic-26349
https://www.auto-data.net/en/audi-s3-sportback-8pa-2.0-tfsi-265hp-quattro-4906
https://www.auto-data.net/en/audi-s3-sportback-8pa-2.0-tfsi-256hp-q

https://www.auto-data.net/en/audi-s8-d5-facelift-2021-4.0-tfsi-v8-571hp-mhev-quattro-tiptronic-45184
https://www.auto-data.net/en/audi-s8-d5-4.0-tfsi-v8-571hp-mhev-quattro-tiptronic-cod-38047
https://www.auto-data.net/en/audi-s8-d4-facelift-2013-4.0-tfsi-cod-v8-plus-605hp-quattro-tiptronic-22456
https://www.auto-data.net/en/audi-s8-d4-facelift-2013-4.0-tfsi-cod-v8-520hp-quattro-tiptronic-20426
https://www.auto-data.net/en/audi-s8-d4-4.0-tfsi-v8-520hp-quattro-tiptronic-19144
https://www.auto-data.net/en/audi-s8-d3-facelift-2007-5.2-fsi-v10-450hp-quattro-tiptronic-27235
https://www.auto-data.net/en/audi-s8-d3-5.2-fsi-v10-450hp-quattro-tiptronic-4789
https://www.auto-data.net/en/audi-s8-d2-4.2-v8-360hp-quattro-tiptronic-4591
https://www.auto-data.net/en/audi-s8-d2-4.2-v8-360hp-quattro-4791
https://www.auto-data.net/en/audi-s8-d2-4.2-v8-340hp-quattro-4790
https://www.auto-data.net/en/audi-sq2-2.0-tfsi-300hp-quattro-s-tronic-35095
https://www.auto-data.net/en/audi-sq5-sportback-fy-3.0-tdi-v

https://www.auto-data.net/en/audi-tt-roadster-8j-2.0-tdi-170hp-quattro-17875
https://www.auto-data.net/en/audi-tt-roadster-8j-1.8-tfsi-160hp-17874
https://www.auto-data.net/en/audi-tt-coupe-8j-3.2i-v6-24v-250hp-quattro-s-tronic-4881
https://www.auto-data.net/en/audi-tt-coupe-8j-3.2i-v6-24v-250hp-quattro-4880
https://www.auto-data.net/en/audi-tt-coupe-8j-2.0-tfsi-200hp-s-tronic-4879
https://www.auto-data.net/en/audi-tt-coupe-8j-2.0-tfsi-200hp-quattro-s-tronic-4878
https://www.auto-data.net/en/audi-tt-coupe-8j-2.0-tfsi-200hp-4877
https://www.auto-data.net/en/audi-tt-coupe-8j-2.0-tdi-170hp-quattro-4876
https://www.auto-data.net/en/audi-tt-coupe-8j-1.8-tfsi-160hp-4875
https://www.auto-data.net/en/audi-tt-roadster-8n-facelift-2000-3.2i-v6-24v-250hp-quattro-automatic-25857
https://www.auto-data.net/en/audi-tt-roadster-8n-facelift-2000-3.2i-v6-24v-250hp-quattro-4874
https://www.auto-data.net/en/audi-tt-roadster-8n-facelift-2000-1.8-t-225hp-quattro-4873
https://www.auto-data.net/en/audi-tt-roa

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [64]:
dicto['e'][dicto['e'].index.duplicated() == True]

,Unnamed: 0,brands_url,models,models_url,generations,generations_url,modifications,modifications_url


In [181]:
#test1 = specifications_of_each_car(dicto['a']['modifications_url'][0])
#test2 = specifications_of_each_car(dicto['a']['modifications_url'][1])
#print(test1,test2)
#type(test1)
#tttt = pd.concat([tttt,test2], ignore_index=True)
#tttt
#df = pd.concat([df,specs3], ignore_index=True)

In [188]:
#Looping to create the final dataframe

#for i in range(0,10):
#    print(dicto['a']['modifications_url'][i])
#dicto['b'].reset_index()
#dicto['a']#['modifications_url'][21]#.drop(columns = ['level_0','index'],inplace = True)
#for i in range(0,(dicto['b']['modifications_url'].shape[0])+1):
 #   print(dicto['b']['modifications_url'][i])
#print(dicto['b']['modifications_url'][29])
dicto['b']['modifications_url'][0]

'https://www.auto-data.net/en/b.engineering-edonis-3.8-v12-690hp-37356'

In [124]:
# #This is the dataframe with all the information we want
# page4 = 'https://www.auto-data.net/en/alfa-romeo-mito-1.6-jtdm-120hp-16680'
# what = pd.read_html(page4)[1]  
# what

# what2 = what[(what['General information'] == 'Brand')
# |(what['General information'] == 'Model')
# |(what['General information'] == 'Generation')
# |(what['General information'] == 'Modification (Engine)')
# |(what['General information'] == 'CO2 emissions') 
# |(what['General information'] == 'Body type')
# |(what['General information'] == 'Fuel tank capacity')
# |(what['General information'] == 'Fuel consumption (economy) - combined')
# |(what['General information'] == 'Fuel Type')
# |(what['General information'] == 'Power')]
# what2.index = what2['General information']
# what3 = what2.T
# what3.drop(axis = 0, labels = 'General information', inplace = True)
# what3['Fuel tank(l)'] = what3['Fuel tank capacity'].apply(lambda x : x.split(' ')[0])
# what3['Fuel consumption(l/100km)'] = what3['Fuel consumption (economy) - combined'].apply(lambda x : x.split(' ')[0])
# what3['Power(Hp)'] = what3['Power'].apply(lambda x: x.split(' ')[0])
# what3['Range (Km)'] = what3[['Fuel tank(l)','Fuel consumption(l/100km)']].apply(lambda item: int(item[0])/float(item[1])*100,axis=1)

# what4 = what3
# what4.drop(columns = ['Fuel consumption (economy) - combined', 'Fuel tank capacity'], inplace = True)
# what4                               #We need to remove this General information column still

In [41]:
#To re-build the dicto from letter.csv file already saved
letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','z']
#for letter in letters:
 #   link = "../BEEV/letter_csvs/"+ letter +".csv"
dicto = {letter : pd.read_csv("../BEEV/letter_csvs/"+ letter +".csv",index_col=None) for letter in letters}
dicto
#pd.DataFrame(a['dataframe'][0])

{'a':       Unnamed: 0                                         brands_url  \
 0              0      https://www.auto-data.net/en/abarth-brand-200   
 1              1      https://www.auto-data.net/en/abarth-brand-200   
 2              2      https://www.auto-data.net/en/abarth-brand-200   
 3              3      https://www.auto-data.net/en/abarth-brand-200   
 4              4      https://www.auto-data.net/en/abarth-brand-200   
 ...          ...                                                ...   
 3870        3870  https://www.auto-data.net/en/autobianchi-brand-51   
 3871        3871  https://www.auto-data.net/en/autobianchi-brand-51   
 3872        3872  https://www.auto-data.net/en/autobianchi-brand-51   
 3873        3873  https://www.auto-data.net/en/autobianchi-brand-51   
 3874        3874  https://www.auto-data.net/en/autobianchi-brand-51   
 
                             models  \
 0     abarth-124-spider-model-2152   
 1     abarth-124-spider-model-2152   
 2     abart

In [46]:
#dicto['b'].drop(columns = 'Unnamed: 0',inplace = True)
dicto['b'].drop(columns = 'index', inplace=True)

In [52]:
dicto['c'].drop(columns = 'Unnamed: 0', inplace = True)
dicto['c']

,brands_url,models,models_url,generations,generations_url,modifications,modifications_url
0,https://www.auto-data.net/en/cadillac-brand-121,cadillac-allante-model-1235,https://www.auto-data.net/en/cadillac-allante-...,cadillac-allante-generation-2474,https://www.auto-data.net/en/cadillac-allante-...,cadillac-allante-4.6-i-v8-32v-299hp-11676,https://www.auto-data.net/en/cadillac-allante-...
1,https://www.auto-data.net/en/cadillac-brand-121,cadillac-allante-model-1235,https://www.auto-data.net/en/cadillac-allante-...,cadillac-allante-generation-2474,https://www.auto-data.net/en/cadillac-allante-...,cadillac-allante-4.5-i-v8-204hp-11675,https://www.auto-data.net/en/cadillac-allante-...
2,https://www.auto-data.net/en/cadillac-brand-121,cadillac-ats-model-2166,https://www.auto-data.net/en/cadillac-ats-mode...,cadillac-ats-coupe-generation-5068,https://www.auto-data.net/en/cadillac-ats-coup...,cadillac-ats-coupe-v-3.6-v6-471hp-automatic-30660,https://www.auto-data.net/en/cadillac-ats-coup...
3,https://www.auto-data.net/en/cadillac-brand-121,cadillac-ats-model-2166,https://www.auto-data.net/en/cadillac-ats-mode...,cadillac-ats-coupe-generation-5068,https://www.auto-data.net/en/cadillac-ats-coup...,cadillac-ats-coupe-v-3.6-v6-471hp-30214,https://www.auto-data.net/en/cadillac-ats-coup...
4,https://www.auto-data.net/en/cadillac-brand-121,cadillac-ats-model-2166,https://www.auto-data.net/en/cadillac-ats-mode...,cadillac-ats-coupe-generation-5068,https://www.auto-data.net/en/cadillac-ats-coup...,cadillac-ats-coupe-v-3.6-v6-461hp-automatic-25988,https://www.auto-data.net/en/cadillac-ats-coup...
...,...,...,...,...,...,...,...
2759,https://www.auto-data.net/en/cupra-brand-256,cupra-leon-model-2831,https://www.auto-data.net/en/cupra-leon-model-...,cupra-leon-generation-7952,https://www.auto-data.net/en/cupra-leon-genera...,cupra-leon-1.4-tsi-245hp-e-hybrid-dsg-41496,https://www.auto-data.net/en/cupra-leon-1.4-ts...
2760,https://www.auto-data.net/en/cupra-brand-256,cupra-tcr-model-2529,https://www.auto-data.net/en/cupra-tcr-model-2529,cupra-tcr-generation-6587,https://www.auto-data.net/en/cupra-tcr-generat...,cupra-tcr-2.0-tsi-350hp-seq-34603,https://www.auto-data.net/en/cupra-tcr-2.0-tsi...
2761,https://www.auto-data.net/en/cupra-brand-256,cupra-tcr-model-2529,https://www.auto-data.net/en/cupra-tcr-model-2529,cupra-tcr-generation-6587,https://www.auto-data.net/en/cupra-tcr-generat...,cupra-tcr-2.0-tsi-350hp-dsg-34604,https://www.auto-data.net/en/cupra-tcr-2.0-tsi...
2762,https://www.auto-data.net/en/cupra-brand-256,cupra-urbanrebel-model-3018,https://www.auto-data.net/en/cupra-urbanrebel-...,cupra-urbanrebel-concept-generation-8618,https://www.auto-data.net/en/cupra-urbanrebel-...,NaN,https://www.auto-data.net/en/nan


In [94]:
#df = dicto['a']['dataframe'][0]
#for i in range(1,dicto['a']['dataframe'].shape[0]+1):
 #   df = pd.concat(df,dicto['a']['dataframe'][i])
#df
#pd.concat([dicto['a']['dataframe'][0].reset_index(drop = True),dicto['a']['dataframe'][1]].reset_index(drop = True))
testdf1 = dicto['a']['dataframe'][0].reset_index(drop = True)
testdf2 = dicto['a']['dataframe'][1].reset_index(drop = True)
pd.concat([testdf1,testdf2],ignore_index = True)


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [85]:
type(dicto['a']['dataframe'][2])

pandas.core.frame.DataFrame

In [80]:
dicto['a']['dataframe'].shape[0]

3875

In [100]:
testdf2

,Brand,Model,Generation,Modification (Engine),Start of production,Powertrain Architecture,Body type,Seats,Doors,Performance specs,...,Front suspension,Rear suspension,Front brakes,Rear brakes,Assisting systems,Steering type,Power steering,Tires size,Wheel rims size,(adsbygoogle = window.adsbygoogle || []).push({});
0,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp),2018 year,Internal Combustion engine,Roadster,2,2,Performance specs,...,Double wishbone,Multi-link independent,Ventilated discs,Disc,ABS (Anti-lock braking system),Steering rack and pinion,Electric Steering,205/45 R17 W84,17,(adsbygoogle = window.adsbygoogle || []).push(...
